In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re
import seaborn as sns


In [60]:
# Import the data
df1 = pd.read_csv("../data/artist_data.csv")  #..(relative path, goes to the parent directory) vs. C:Users (absolute path)
df2 = pd.read_csv("../data/artwork_data.csv")

C:\Users\jenny\AppData\Local\Temp\ipykernel_30792\2521364596.py:3: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../data/artwork_data.csv")


In [68]:
# Select the desired columns from df1
df1_selected = df1[['id', 'name', 'yearOfBirth', 'gender']]

# Select the desired columns from df2
df2_selected = df2[['artistId', 'year', 'acquisitionYear']]

# Merge the selected columns based on artistId
merged_df = pd.merge(df1_selected, df2_selected, left_on='id', right_on='artistId')

# Select the final columns for the new data frame
result_df = merged_df[['name', 'yearOfBirth', 'gender', 'year', 'acquisitionYear']]

# Display the resulting data frame
print(result_df)

                         name  yearOfBirth  gender    year  acquisitionYear
0      Abakanowicz, Magdalena       1930.0  Female    1978           2009.0
1      Abakanowicz, Magdalena       1930.0  Female    1969           2009.0
2      Abakanowicz, Magdalena       1930.0  Female    1971           2009.0
3      Abakanowicz, Magdalena       1930.0  Female    1976           2009.0
4         Abbey, Edwin Austin       1852.0    Male  1882.0           1924.0
...                       ...          ...     ...     ...              ...
69190         Zorio, Gilberto       1944.0    Male    1971           2009.0
69191              Zox, Larry       1936.0    Male  1965.0           1968.0
69192   Zuccarelli, Francesco       1702.0    Male  1765.0           1985.0
69193        Zuloaga, Ignacio       1870.0    Male  1905.0           1923.0
69194         Zyw, Aleksander       1905.0    Male  1957.0           1962.0

[69195 rows x 5 columns]


In [69]:
# Check if any 'yearOfBirth' values are greater than 'acquisitionYear'
inconsistent_data = result_df[result_df['yearOfBirth'] > result_df['acquisitionYear']]

# Display the inconsistent data, if any
if len(inconsistent_data) > 0:
    print("Inconsistent data found:")
    print(inconsistent_data)
else:
    print("No inconsistent data found. All 'yearOfBirth' values are less than 'acquisitionYear'.")

No inconsistent data found. All 'yearOfBirth' values are less than 'acquisitionYear'.


In [70]:
# Convert 'year' column to numeric
result_df['year'] = pd.to_numeric(result_df['year'], errors='coerce')

# Find the minimum year for each artist
unique_df = result_df.groupby('name').agg({'year': 'min'}).reset_index()

# Merge with the original dataframe to retrieve the additional columns
unique_df = pd.merge(unique_df, result_df, on=['name', 'year'], how='left')

# Display the resulting dataframe
print(unique_df)

                        name    year  yearOfBirth  gender  acquisitionYear
0     Abakanowicz, Magdalena  1969.0       1930.0  Female           2009.0
1        Abbey, Edwin Austin  1882.0       1852.0    Male           1924.0
2           Abbott, Berenice  1935.0       1898.0  Female           2010.0
3     Abbott, Lemuel Francis     NaN       1760.0    Male           1885.0
4     Abbott, Lemuel Francis     NaN       1760.0    Male           1968.0
...                      ...     ...          ...     ...              ...
6155         Zyw, Aleksander  1957.0       1905.0    Male           1962.0
6156         di Suvero, Mark  1976.0       1933.0    Male           2004.0
6157            van Elk, Ger  1976.0       1941.0    Male           1980.0
6158       Štyrský, Jindrich  1934.0       1899.0    Male           2007.0
6159       Štyrský, Jindrich  1934.0       1899.0    Male           2007.0

[6160 rows x 5 columns]


C:\Users\jenny\AppData\Local\Temp\ipykernel_30792\2555363441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['year'] = pd.to_numeric(result_df['year'], errors='coerce')


In [71]:
# Find the earliest acquisition year for each unique name
filtered_df = unique_df.groupby('name').agg({'acquisitionYear': 'min'}).reset_index()

# Merge with the original dataframe to retrieve the additional columns
filtered_df = pd.merge(filtered_df, unique_df, on=['name', 'acquisitionYear'], how='left')

# Display the resulting dataframe
print(filtered_df)

                        name  acquisitionYear    year  yearOfBirth  gender
0     Abakanowicz, Magdalena           2009.0  1969.0       1930.0  Female
1        Abbey, Edwin Austin           1924.0  1882.0       1852.0    Male
2           Abbott, Berenice           2010.0  1935.0       1898.0  Female
3     Abbott, Lemuel Francis           1885.0     NaN       1760.0    Male
4             Abrahams, Ivor           1986.0  1967.0       1935.0    Male
...                      ...              ...     ...          ...     ...
5807         Zyw, Aleksander           1962.0  1957.0       1905.0    Male
5808         di Suvero, Mark           2004.0  1976.0       1933.0    Male
5809            van Elk, Ger           1980.0  1976.0       1941.0    Male
5810       Štyrský, Jindrich           2007.0  1934.0       1899.0    Male
5811       Štyrský, Jindrich           2007.0  1934.0       1899.0    Male

[5812 rows x 5 columns]


In [72]:
# Drop duplicate names and keep only the first occurrence
unique_names_df = filtered_df.drop_duplicates(subset='name', keep='first')

# Display the resulting dataframe
print(unique_names_df)

                        name  acquisitionYear    year  yearOfBirth  gender
0     Abakanowicz, Magdalena           2009.0  1969.0       1930.0  Female
1        Abbey, Edwin Austin           1924.0  1882.0       1852.0    Male
2           Abbott, Berenice           2010.0  1935.0       1898.0  Female
3     Abbott, Lemuel Francis           1885.0     NaN       1760.0    Male
4             Abrahams, Ivor           1986.0  1967.0       1935.0    Male
...                      ...              ...     ...          ...     ...
5806        Zuloaga, Ignacio           1923.0  1905.0       1870.0    Male
5807         Zyw, Aleksander           1962.0  1957.0       1905.0    Male
5808         di Suvero, Mark           2004.0  1976.0       1933.0    Male
5809            van Elk, Ger           1980.0  1976.0       1941.0    Male
5810       Štyrský, Jindrich           2007.0  1934.0       1899.0    Male

[3332 rows x 5 columns]


In [73]:
# Calculate the year difference
unique_names_df['yearDifference'] = unique_names_df['acquisitionYear'] - unique_names_df['yearOfBirth']

# Display the resulting dataframe
print(unique_names_df)

                        name  acquisitionYear    year  yearOfBirth  gender   
0     Abakanowicz, Magdalena           2009.0  1969.0       1930.0  Female  \
1        Abbey, Edwin Austin           1924.0  1882.0       1852.0    Male   
2           Abbott, Berenice           2010.0  1935.0       1898.0  Female   
3     Abbott, Lemuel Francis           1885.0     NaN       1760.0    Male   
4             Abrahams, Ivor           1986.0  1967.0       1935.0    Male   
...                      ...              ...     ...          ...     ...   
5806        Zuloaga, Ignacio           1923.0  1905.0       1870.0    Male   
5807         Zyw, Aleksander           1962.0  1957.0       1905.0    Male   
5808         di Suvero, Mark           2004.0  1976.0       1933.0    Male   
5809            van Elk, Ger           1980.0  1976.0       1941.0    Male   
5810       Štyrský, Jindrich           2007.0  1934.0       1899.0    Male   

      yearDifference  
0               79.0  
1               7

C:\Users\jenny\AppData\Local\Temp\ipykernel_30792\3456388171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_names_df['yearDifference'] = unique_names_df['acquisitionYear'] - unique_names_df['yearOfBirth']
